In [1]:
# !pip install asyncio
# !pip install nest-asyncio
# !pip install pandas

In [15]:
# API_KEY = "zgJAdEg3lkVxGXyEtM89tOZplOsTi8bv" # Personal
API_KEY = "pqmKVrIjJjkKQMhvRslPapP7QzNV2A1I"# Hackathon
notebook_dir = "C:\\Users\\William\\Documents\\ML\\a16z Hackathon\\A16ZxMistral\\single_image_test"
image_foler = "images"
images_dir = f"{notebook_dir}/{image_foler}"

In [16]:
import os
from mistralai import Mistral
import base64
import asyncio
import nest_asyncio
nest_asyncio.apply()
import time

model = "pixtral-12b-latest"
client = Mistral(api_key=API_KEY)

In [17]:
def encode_image_base64(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode("utf-8")



In [18]:
files = os.listdir(images_dir)
encoded_images = [encode_image_base64(f"{images_dir}/{file}") for file in files]
prompts = [
	{
		"name": "fire",
		"prompt": "Do you see fire in this image? If so return 'True' otherwise return 'False' without the quotation marks."
	},
	{
		"name": "smoke",
		"prompt": "Do you see smoke or dense steam in the iamge ? If so return 'True' otherwise return 'False' without the quotation marks."
	},
	{
		"name": "police",
		"prompt": "Do you see police or law enforcement present in the image? If so return 'True' otherwise return 'False' without the quotation marks."
	},
	{
		"name": "Formations",
		"prompt": "Do you see people orginised in formations e.g. standing in organised lines? If so return 'True' otherwise return 'False' without the quotation marks."
	},
	{
		"name": "crowds",
		"prompt": "Do you see any crowds of people of 20 or more? If so return 'True' otherwise return 'False' without the quotation marks."
	},
	{
		"name": "crush",
		"prompt": "Do you see any crowds of more than 60 people, that could be dangerous to vunerable people? If so return 'True' otherwise return 'False' without the quotation marks."
	},
]



In [22]:

async def get_pixtral_reponse(encoded_image, prompt):
    print(f"Prompt: {prompt['name']} {prompt['prompt'][:30]}")
    try_number = 0
    while True: 
        try:
            start = time.time()
            chat_response = client.chat.complete(
                model=model,
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": f"{prompt['prompt']}"
                            },
                            {
                                "type": "image_url",
                                "image_url": f"data:image/png;base64,{encoded_image}"
                            },
                        ]
                    },
                ]
            )
            response = chat_response.choices[0].message.content
            end = time.time()
            print(f"Attempt: {try_number} Time {end - start:.2f}s")
            break
        except: 
            continue
    return response

async def analyse_image(encoded_image, prompts):
    coroutines = [
        get_pixtral_reponse(encoded_image, prompt) 
        for prompt in prompts
    ]
    results = await asyncio.gather(*coroutines)
    return results

async def analyse_images(encoded_images, prompts):
    coroutines = [
        analyse_image(encoded_image, prompts) 
        for encoded_image in encoded_images
    ]
    results = await asyncio.gather(*coroutines)
    return results

In [23]:
single_result = await get_pixtral_reponse(encoded_images[0], prompts[0])
print(single_result )

Prompt: fire Do you see fire in this image?
Request: 0 Time 2.20s
False


In [24]:

output = asyncio.run(analyse_image(encoded_images[0], prompts))

Prompt: fire Do you see fire in this image?
Request: 0 Time 2.08s
Prompt: smoke Do you see smoke or dense stea
Request: 0 Time 1.69s
Prompt: police Do you see police or law enfor
Request: 0 Time 1.82s
Prompt: Formations Do you see people orginised in


In [13]:
output

NameError: name 'output' is not defined

In [ ]:

final_results = asyncio.run(analyse_images(encoded_images, prompts))



Prompt: fire
Prompt: smoke
Prompt: police
Prompt: Formations
Prompt: crowds
Prompt: crush
Prompt: fire
Prompt: smoke
Prompt: police
Prompt: Formations
Prompt: crowds
Prompt: crush
Prompt: fire
Prompt: smoke
Prompt: police
Prompt: Formations
Prompt: crowds
Prompt: crush
Prompt: fire
Prompt: smoke
Prompt: police
Prompt: Formations
Prompt: crowds
Prompt: crush
Prompt: fire
Prompt: smoke
Prompt: police
Prompt: Formations
Prompt: crowds
Prompt: crush
Prompt: fire
Prompt: smoke
Prompt: police
Prompt: Formations
Prompt: crowds
Prompt: crush
Prompt: fire
Prompt: smoke
Prompt: police
Prompt: Formations
Prompt: crowds
Prompt: crush
Prompt: fire
Prompt: smoke
Prompt: police
Prompt: Formations
Prompt: crowds
Prompt: crush


RemoteProtocolError: Server disconnected without sending a response.

In [ ]:
# final_results
columns = [prompt["name"] for prompt in prompts]
rows = [file.split(".")[0] for file in files]

import pandas as pd

df = pd.DataFrame(final_results, columns=columns, index=rows)
print(df)


NameError: name 'prompts' is not defined